# Title: Data cleaning

## Description:
In this file we are going to load the open data of the trees under maintenance in Bologna and clean it to be able to use it in i-Tree Eco.

Mainly we are going to:
- Load the data from the opendata website
- Clean the data
    - Calculate the diameter at breast height (dbh)
    - Calculate the height of each tree
    - Split the "GeoPoint" column into latitude and longitude
    - Calculate the crown size from the species name and height
- Save the data in a xlsx file

The data will be then used in the i-Tree Eco model to calculate the ecobenefit provided by the trees.
The resulting data will be managed in the file dataModeling.ipynb
[]: # Path: dataModeling.ipynb

In [1]:
import pandas as pd
import numpy as np
import re
from fuzzywuzzy import fuzz

/Users/luca/opt/anaconda3/envs/geo_env/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
bologna_trees_url= "https://opendata.comune.bologna.it/api/explore/v2.1/catalog/datasets/alberi-manutenzioni/exports/csv?lang=it&timezone=Europe%2FRome&use_labels=true&delimiter=%3B"
bologna_trees = pd.read_csv(bologna_trees_url, sep=";")

In [3]:
# print the number of rows and columns
print(bologna_trees.shape)

(85043, 19)


In [3]:
bologna_trees['Classe circonferenza (diametro)'].unique()

array(['Cl2: 15 - 30 (5-10 cm)', 'Cl1: < 15 (5 cm)',
       'Cl6: 90 - 110 (28-35cm)', 'Cl7: 110 - 140 (35-45 cm)',
       'Cl3: 30 - 45 (10 – 15 cm)', 'Cl5: 60 - 90 (19-28 cm)',
       'Cl8: 140 - 170 (45-54cm)', 'Cl11: 230 - 260 (73-80 cm)',
       'Cl12: > 260 (> 80 cm)', 'Cl4: 45 - 60 (15-19 cm)',
       'Cl9: 170 - 200 (54-64 cm)', 'Cl10: 200 - 230 (64-73 cm)', nan],
      dtype=object)

In [4]:
# count the values that have nan is the column 'Classe circonferenza (diametro)'
bologna_trees['Classe circonferenza (diametro)'].isna().sum()

44

In [5]:
# Remove the 'Classe circonferenza (diametro)' column and create a new 'dbh' column, with the diameter at breast height
def convert_diameter(value):
    if isinstance(value, str):
        parts = value.split(':')
        if len(parts) == 2:
            range_values = parts[1].strip().split('-')
            if len(range_values) == 3:
                min_value = re.search(r'\((\d+)', range_values[1])
                min_value = min_value.group(1)
                max_value = ''.join(filter(str.isdigit, range_values[2]))
                if min_value.isdigit() and max_value.isdigit():
                    min_value = int(min_value)
                    max_value = int(max_value)
                    return (min_value + max_value) / 2  # Taking the average+
            else: 
                # that means is of class 'Cl12: > 260 (> 80 cm)
                return 80
    return None


# Apply the conversion function to create a new 'dbh' column
bologna_trees['dbh'] = bologna_trees['Classe circonferenza (diametro)'].apply(convert_diameter)

In [6]:
# find the unique values of the 'Classe di altezza' column
bologna_trees['Classe di altezza'].unique()

array(['Cl1: <6mt', 'Cl4: 16mt - 23mt', 'Cl2: 6mt - 12mt',
       'Cl3: 12mt - 16mt', 'Cl5: >23mt', nan, 'ClCl1: <6mt: <6mt'],
      dtype=object)

In [7]:
# Define a custom function to extract the mean height value
def extract_mean_height_Bologna(value):
    if isinstance(value, str):
        parts = value.split(':')
        if len(parts) == 2:
            range_values = parts[1].strip().split('-')
            if len(range_values) == 2:
                min_value = float(range_values[0].strip()[:-2])  # Removing 'mt'
                max_value = float(range_values[1].strip()[:-2])  # Removing 'mt'
            # if we have the Cl 1 (Cl1: <6mt), put the heigth to 3
            else:
                min_value = 3
                max_value = 3
            return (min_value + max_value) / 2  # Taking the average
    return None

# Apply the custom function to create a new 'total_height' column
bologna_trees['total_height'] = bologna_trees['Classe di altezza'].apply(extract_mean_height_Bologna)

In [8]:
# Split the "GeoPoint" column into latitude and longitude
bologna_trees[['Latitude', 'Longitude']] = bologna_trees['Geo Point'].str.split(',', expand=True).astype(float)

# Title: Crown size
## Description:
Due to the problem that crown size is a reccomended field in i-Tree Eco we want to find a way to estimate it from other parameters. 
## Method:
- We searched for datasets:
    - In the italian government website: https://www.dati.gov.it/view-dataset?Cerca=alberi&page=1
    - In the european website: https://data.europa.eu/data/datasets/c91a4635-8b7d-43fe-9b27-d95dec8392a7?locale=en
- For these datasets we searched for the field crown size and for the ones that had it we downloaded them.
- After that we matched the species inside our dataset with the ones in the downloaded datasets.

In [ ]:
vienna_trees = pd.read_csv('../data/crown_size/alberi_Vienna.csv', sep=",")

In [9]:
# rename the columns to the italian names
vienna_trees.rename(columns={'OBJECTID':'ID OGGETTO'}, inplace=True)
vienna_trees.rename(columns={'SHAPE':'FORMA'}, inplace=True)
vienna_trees.rename(columns={'BAUM_ID':'ID_ALBERO'}, inplace=True)
vienna_trees.rename(columns={'DATENFUEHRUNG':'GUIDA DATI'}, inplace=True)
vienna_trees.rename(columns={'BEZIRK':'DISTRICT'}, inplace=True)
vienna_trees.rename(columns={'OBJEKT_STRASSE':'OBJECT_STREET'}, inplace=True)
vienna_trees.rename(columns={'GEBIETSGRUPPE':'AREA GRUPPO'}, inplace=True)
vienna_trees.rename(columns={'PFLANZJAHR':'ANNO DI PIANTATURA'}, inplace=True)
vienna_trees.rename(columns={'PFLANZJAHR_TXT':'ANNO_TXT PIANTATURA'}, inplace=True)
vienna_trees.rename(columns={'STAMMUMFANG':'CIRCONFERENZA TRUN'}, inplace=True)
vienna_trees.rename(columns={'STAMMUMFANG_TXT':'CIRCUMFERENZA FIDUCIA_TXT'}, inplace=True)
vienna_trees.rename(columns={'BAUMHOEHE':'ALTEZZA ALBERO'}, inplace=True)
vienna_trees.rename(columns={'BAUMHOEHE_TXT':'ALTEZZA ALBERO_TXT'}, inplace=True)
vienna_trees.rename(columns={'KRONENDURCHMESSER':'DIAMETRO CORONA'}, inplace=True)
vienna_trees.rename(columns={'KRONENDURCHMESSER_TXT':'DIAMETRO CORONA_TXT'}, inplace=True)
vienna_trees.rename(columns={'BAUMNUMMER':'NUMERO ALBERO'}, inplace=True)

# change the column "GATTUNG_ART" in vienna_trees to "Species arborea" and change the values inside it, remove the () at the end
# example: Quercus spec. (Eiche) -> Quercus spec.
vienna_trees.rename(columns={'GATTUNG_ART':'Specie arborea'}, inplace=True)
vienna_trees['Specie arborea'] = vienna_trees['Specie arborea'].str.replace(r"\(.*\)","")


/var/folders/j8/fjrnp8w15sbf5q2fdgnpb0br0000gn/T/ipykernel_42984/3146187152.py:22: FutureWarning: The default value of regex will change from True to False in a future version.
  vienna_trees['Specie arborea'] = vienna_trees['Specie arborea'].str.replace(r"\(.*\)","")


We extract the unique values for the column 'Specie arborea' from the different datasets

In [4]:
species_Bologna = bologna_trees['Specie arborea'].unique()
species_Vienna = vienna_trees['Specie arborea'].unique()

In [6]:
species_Bologna.shape

(315,)

In [11]:
# print a sample of the species in Bologna
species_Bologna[0:5]

array(['Prunus cerasifera (var. Pissardi)', 'Fraxinus ornus',
       'Fraxinus excelsior', 'Acer campestre', 'Aesculus hippocastanum'],
      dtype=object)

In [12]:
# print a sample of the species in Vienna
species_Vienna[0:5]

array(["Acer platanoides 'Olmstedt' ", "Tilia cordata 'Rancho' ",
       "Acer campestre 'Elsrijk' ", 'Populus x canadensis ',
       'Acer platanoides '], dtype=object)

In [13]:
# print the unique values of DIAMETRO CORONA_TXT for the species 'Populus x canadensis ' (Vienna)
vienna_trees.loc[vienna_trees['Specie arborea'] == 'Populus x canadensis ', 'DIAMETRO CORONA_TXT'].unique()

array(['13-15 m', '19-21 m', '7-9 m', '16-18 m', '10-12 m', '4-6 m',
       '0-3 m', '>21 m', 'nicht bekannt'], dtype=object)

Now we write a function that compares the species in our dataset and return the best match for each one of them in the downloaded datasets.
For the string similarity we use the fuzzywuzzy library. Fuzzy string matching like a boss. It uses Levenshtein Distance to calculate the differences between sequences in a simple-to-use package.

In [14]:
# Convert all elements to strings
species_Bologna = np.array([str(item) for item in species_Bologna], dtype=object)
species_Vienna = np.array([str(item) for item in species_Vienna], dtype=object)

# Initialize lists to store results
result_data = []
no_matching_species = []

# Iterate through each species in array_1
for species_bol in species_Bologna:
    best_match = None
    best_score = 0

    # Calculate similarity scores with species in array_2
    for species_v in species_Vienna:
        similarity_score = fuzz.token_sort_ratio(species_bol, species_v)
        if similarity_score > best_score:
            best_score = similarity_score
            best_match = species_v

    # Check if the similarity score is below 70, we just pick the family name of the species
    if best_score < 70:
        # Check if the first word of species_bol matches any first word in array_2
        first_word_1 = species_bol.split()[0]
        matching_family = None

        # Iterate through the array
        for item in species_Vienna:
            if first_word_1 in item:
                matching_family = item
                break  # Exit the loop as soon as a match is found
        
        if matching_family:
            # Include records with a family match in result_data
            result_data.append({'Species_Bologna': species_bol, 'Closest_Species': best_match, 'Similarity_Score': best_score})
        else:
            # Save records with no matching family in no_matching_species
            no_matching_species.append({'Species_Bologna': species_bol, 'Closest_Species': '', 'Similarity_Score': 0})
    else:
        # Include records with a similarity score above or equal to 70 in result_data
        result_data.append({'Species_Bologna': species_bol, 'Closest_Species': best_match, 'Similarity_Score': best_score})

# Create a DataFrame from the lists of dictionaries
result_df = pd.DataFrame(result_data)
no_matching_species_df = pd.DataFrame(no_matching_species)

To summarize, we will have:
- result_df: the dataframe with the species in our dataset and the best match for each one of them in the downloaded datasets. The ones with a similarity score lower than 70 are the ones with the family name in common (for family name we consider the first word of the species, example: "Quercus spec. (Eiche)" -> "Quercus" is the family name).
- no_matching_species_df: the dataframe with the species in our dataset that had a similarity score lower than 70 with all the species in the downloaded datasets and no family name in common.

Here we can see the results of the function and the species that have been matched.

In [15]:
# find the unique values for the family names of the no matching species
no_matching_species_df['Family'] = no_matching_species_df['Species_Bologna'].str.split().str[0]
no_matching_species_df['Family'].unique()

array(['Lagestroemia', 'Olea', 'Zyziphus', 'Melia', 'Diospyros', 'Laurus',
       'Sterculia', 'Libocedrus', 'Ligustrum', 'Eriobotrya', 'nan',
       'Punica', 'Kalmia', 'Pyracantha', 'Arbutus', 'Forsythia', 'Acacia',
       'Bignonia', 'Osmanthus', 'Nerium', 'Eleagnus', 'Chimonanthus',
       'Pittosporum', 'Chamaerops', 'Viburnum', 'Yucca', 'Chaenomeles',
       'Trachicarpus', 'Washingtonia', 'Vite', 'Vitex', 'Phyllirea',
       'Actinidia', 'Deutzia'], dtype=object)

In [16]:
# Clean the column 'DIAMETRO CORONA_TXT' in vienna_trees

# drop the rows where the value is 'nicht bekannt' in vienna_trees['DIAMETRO CORONA_TXT']
vienna_trees = vienna_trees[vienna_trees['DIAMETRO CORONA_TXT'] != 'nicht bekannt']

# if the value is '>21 ' replace it with 21
vienna_trees['DIAMETRO CORONA_TXT'] = vienna_trees['DIAMETRO CORONA_TXT'].str.replace('>21 ','21')

# the column 'DIAMETRO CORONA_TXT' contains values like: 0-3 m, 13-15 m, etc.. I want to:
# 1) remove the 'm' at the end of the string
# 2) split the string and take the mean of the two values
# 3) convert the string to float
vienna_trees['DIAMETRO CORONA_TXT'] = vienna_trees['DIAMETRO CORONA_TXT'].str.replace('m','')
vienna_trees['DIAMETRO CORONA_TXT'] = vienna_trees['DIAMETRO CORONA_TXT'].str.split('-').apply(lambda x: (float(x[0]) + float(x[1])) / 2 if len(x) == 2 else float(x[0]))
vienna_trees['DIAMETRO CORONA_TXT'] = vienna_trees['DIAMETRO CORONA_TXT'].astype(float)

In [17]:
vienna_trees['ALTEZZA ALBERO_TXT'].unique()

array(['6-10 m', '0-5 m', '26-30 m', '11-15 m', '16-20 m', '21-25 m',
       '> 35 m', 'nicht bekannt', '31-35 m'], dtype=object)

In [18]:
# Define a custom function to extract the mean height value
def extract_mean_height_Vienna(value):
    if isinstance(value, str) and value.strip():
        # Extract the numeric parts
        parts = value.split('-')
        if len(parts) == 2:
            min_value = float(parts[0]) # Convert to float
            max_value = float(parts[1].strip()[:-2])  # Removing ' m'
            return (min_value + max_value) / 2  # Taking the average
        elif value == 'nicht bekannt':
            return np.nan  # Handle 'nicht bekannt' (unknown) as NaN
    # Return the value converted to float
    return float(value)

vienna_trees['ALTEZZA ALBERO_TXT'] = vienna_trees['ALTEZZA ALBERO_TXT'].str.replace('> 35 m','35')
# Apply the custom function to create a new 'total_height' column
vienna_trees['total_height'] = vienna_trees['ALTEZZA ALBERO_TXT'].apply(extract_mean_height_Vienna)

In [19]:
# Map 'Species_Bologna' to 'Closest_Species' in result_df
species_mapping = result_df.set_index('Species_Bologna')['Closest_Species'].to_dict()

# Map 'Species_Bologna' in bologna_trees to 'Closest_Species'
bologna_trees['Closest_Species'] = bologna_trees['Specie arborea'].map(species_mapping)

# Initialize 'crown_width' column with NaN values
bologna_trees['crown_width'] = np.nan

# Iterate through each row in bologna_trees
for index, bologna_row in bologna_trees.iterrows():
    # Find the corresponding species in Vienna dataset (Closest_Species)
    matching_species = bologna_row['Closest_Species']
    
    # Filter Vienna dataset to find matching trees
    matching_trees_vienna = vienna_trees[vienna_trees['Specie arborea'] == matching_species]
    
    # Check if there are matching trees
    if not matching_trees_vienna.empty:
        # Calculate the absolute height difference
        height_diff = np.abs(matching_trees_vienna['total_height'] - bologna_row['total_height'])
        
        # Find the index of the closest tree
        closest_index = height_diff.idxmin()

        # Update 'crown_width' for the current row with the value from Vienna dataset
        # if closest_index is nan, skip
        if pd.isna(closest_index) or matching_trees_vienna.at[closest_index, 'DIAMETRO CORONA_TXT'] == 'nicht bekannt':
            continue
        else:
            bologna_trees.at[index, 'crown_width'] = matching_trees_vienna.at[closest_index, 'DIAMETRO CORONA_TXT']

In [20]:
# count the values where the column 'crown_width' is nan
bologna_trees['crown_width'].isna().sum()

2340

Here we are going to cover manually the species that have not been matched by the function. We will search in internet for the mean crown size of these species and we will add them to the dataset.

In [21]:
# create a dictionary with the mean values of the species, like Lagestroemia: 3.5, Olea: 3.5, etc.., we have to fill it manually
mean_width_species = {'Lagestroemia': 3.5, 'Olea': 2, 'Zyziphus': 7, 'Melia': 5, 'Diospyros': 6, 'Laurus': 1,
         'Sterculia': 5, 'Libocedrus': 6, 'Ligustrum': 1.5, 'Eriobotrya': 5.5,
         'Punica': 2.5, 'Kalmia': 1, 'Pyracantha': 1.5, 'Arbutus': 4.5, 'Forsythia': 3, 'Acacia': 6.5,
         'Bignonia': 2.5, 'Osmanthus': 8, 'Nerium': 2, 'Eleagnus': 6, 'Chimonanthus': 2.5,
         'Pittosporum': 2.5, 'Chamaerops': 1, 'Viburnum': 3, 'Yucca': 2, 'Chaenomeles': 1.5,
         'Trachicarpus': 1, 'Washingtonia': 1.5, 'Vite': 1, 'Vitex': 4, 'Phyllirea': 1.5,
         'Actinidia': 2, 'Deutzia': 2}

# replace the nan values in the column 'crown_width' with the mean of the values of the species
bologna_trees['crown_width'] = bologna_trees['crown_width'].fillna(bologna_trees['Specie arborea'].map(mean_width_species))

In [23]:
# save the new dataframe to a new excel file
bologna_trees.to_excel('output_file_cleaned.xlsx', index=False)

In [30]:
bologna_trees.shape

(86088, 24)

Possible documentations:
- http://www.fossilworks.org/cgi-bin/bridge.pl?a=taxonInfo&taxon_no=54594
- Trees Vienna: https://data.europa.eu/data/datasets/c91a4635-8b7d-43fe-9b27-d95dec8392a7?locale=en
- Tree eco evaluation: https://github.com/kodujdlapolski/tree-research/blob/master/documents/An%20evaluation%20of%20the%20ecosystem%20services%20provided%20by%20urban%20trees.pdf

Links:
- Messina: https://www.dati.gov.it/view-dataset/dataset?id=e53a56bd-5744-4334-b8fe-c81a8f5aabd4
- Ferrara: https://dati.comune.fe.it/dataset/rilievo-alberature-ferraratua

In [25]:
trees_Messina = pd.read_csv('../data/crown_size/alberi_Messina.csv', sep=",")
trees_Ferrara = pd.read_csv('../data/crown_size/alberi_Ferrara.csv', sep=",")

/var/folders/j8/fjrnp8w15sbf5q2fdgnpb0br0000gn/T/ipykernel_42984/3445865149.py:2: DtypeWarning: Columns (9,10,14,18,31,34,39,40,41,42,43,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  trees_Ferrara = pd.read_csv('../data/crown_size/alberi_Ferrara.csv', sep=",")


In [26]:
print(trees_Ferrara.iloc[0])

FID                     Rilievo_alberature_FerraraTUA_preview.1
the_geom              POINT (707819.73506861 4969150.947868596)
GID                                                           1
ID_FETUA_P                                                    1
LAT                                                    44.84549
LON                                                    11.62973
TP                                                            1
TS                                                            3
CODICE                                               P 1 03 108
ZONA                                                        NaN
UBICAZIONE                                                  NaN
INDIRIZZO                            Via delle Vigne 36 Ferrara
AREA_RISCH                                                 Alto
ISTATCODE                                                 38008
PROTEZIONE                                                  NaN
PIANTA_SPE                              

In [27]:
trees_Ferrara['Family'] = trees_Ferrara['PIANTA_SPE'].str.split().str[0]

match_Ferrara = []
for no_match_family in no_matching_species_df['Family'].unique():
    # check if the family name is in the column 'Family' of trees_Ferrara
    if no_match_family in trees_Ferrara['Family'].unique():
        # add the family name to the list
        match_Ferrara.append(no_match_family)
        # print the family name
        print(no_match_family)

Olea
Melia
Diospyros
Laurus
Libocedrus
Ligustrum
Eriobotrya
Punica
Acacia
Nerium
Viburnum
Yucca


In [28]:
# keep inside the dataset trees_Ferrara just the family names that are in match_Ferrara
trees_Ferrara = trees_Ferrara[trees_Ferrara['Family'].isin(match_Ferrara)]

In [29]:
# Define a custom function to extract the mean height value
def extract_mean_height_Ferrara(value):
    if isinstance(value, str) and value.strip():
        # Extract the numeric parts
        parts = value.split('-')
        if len(parts) == 2:
            min_value = float(parts[0]) # Convert to float
            max_value = float(parts[1].strip()[:-2])  # Removing ' m'
            return (min_value + max_value) / 2  # Taking the average
        elif value == 'nicht bekannt':
            return np.nan  # Handle 'nicht bekannt' (unknown) as NaN
    return np.nan  # Return NaN for empty or invalid values

# Apply the custom function to create a new 'total_height' column
trees_Ferrara['total_height'] = trees_Ferrara[''].apply(extract_mean_height_Ferrara)

KeyError: ''

In [ ]:
# now do the same for trees_Messina
trees_Messina['Family'] = trees_Messina['SPECIE_SC'].str.split().str[0]

match_Messina = []
for no_match_family in no_matching_species_df['Family'].unique():
    # check if the family name is in the column 'Family' of trees_Messina
    if no_match_family in trees_Messina['Family'].unique():
        # add the family name to the list
        match_Messina.append(no_match_family)
        # print the family name
        print(no_match_family)

# keep inside the dataset trees_Messina just the family names that are in match_Messina
trees_Messina = trees_Messina[trees_Messina['Family'].isin(match_Messina)]

Olea
Melia
Laurus
Ligustrum
Eriobotrya
Acacia
Nerium
Pittosporum
Yucca
Washingtonia


In [ ]:
# Define a custom function to extract the mean height value
def extract_mean_height_Messina(value):
    if isinstance(value, str) and value.strip():
        # Extract the numeric parts
        parts = value.split('-')
        if len(parts) == 2:
            min_value = float(parts[0]) # Convert to float
            max_value = float(parts[1].strip()[:-2])  # Removing ' m'
            return (min_value + max_value) / 2  # Taking the average
        elif value == 'nan':
            return np.nan  # Handle 'nicht bekannt' (unknown) as NaN
    # Return the value converted to float
    return float(value)

trees_Messina['ALTEZZA'] = trees_Messina['ALTEZZA'].str.replace('>20 m','20')
# Apply the custom function to create a new 'total_height' column
trees_Messina['total_height'] = trees_Messina['ALTEZZA'].apply(extract_mean_height_Messina)

In [ ]:
#trees_Messina[['ID_SCHEDA', 'CODICE', 'DATA_ORA_M', 'DATA_RILIE', 'DESCRIZION','DIAMETRO', 'HRILIEVO1', 'ALTEZZA', 'ANNO', 'CABIFORCA', 'CACARIE']].head()

#trees_Messina[['CACAVITA', 'CACORTECCI', 'CCARIE', 'CCARPOFORI', 'CCAVITA', 'CFERITE', 'CHAPICE', 'CHASIMME', 'CHBRANCA', 'CHCAPITOZZ', 'CHCARIE']].head()

#trees_Messina[['CHCARPOFOR', 'CHDENSITA', 'CHLIMITI', 'CHRAMIS', 'CHTAGLIPOT','CLIMITI', 'CPC', 'DIMORA', 'DIREZIO1', 'DIREZIO2', 'DIREZIO3']].head()

#trees_Messina[['DIREZIO4', 'DIREZIO5', 'DIREZIO6', 'DIREZIO7', 'DIREZIO8', 'FCARIE','FCARPOFORI', 'FCAVITA', 'FFERITE', 'FGETTI', 'FINCLINAZI', 'INABBA']].head()

#trees_Messina[['INPOTATURA', 'INREIMPIA', 'INSETTI', 'INTECNICA', 'INTIPO','INURGENZA', 'LUOGO', 'POSIZIONE', 'RAFFIORANT', 'RCARIE', 'RCARPOFORI']].head()

#trees_Messina[['RILEVATORE', 'RILIEVO', 'RLIMITI', 'RSTROZZANT', 'SPECIE_SC','SPECIE_VL', 'VIGORIA', 'PROFILO1', 'PROFILO2', 'PROFILO3', 'PROFILO4']].head()

#trees_Messina[['SPECIE_VL', 'VIGORIA', 'PROFILO1', 'PROFILO2', 'PROFILO3', 'PROFILO4','PROFILO5', 'PROFILO6', 'PROFILO7', 'PROFILO8', 'T_ERRE1', 'T_ERRE2','INCLINAZIO', 'HRILIEVO2', 'Family', 'total_height']].head()

,SPECIE_VL,VIGORIA,PROFILO1,PROFILO2,PROFILO3,PROFILO4,PROFILO5,PROFILO6,PROFILO7,PROFILO8,T_ERRE1,T_ERRE2,INCLINAZIO,HRILIEVO2,Family,total_height
3,Albero dei rosari (Melia),sufficiente,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,Melia,13.0
7,Olivo,sufficiente,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Olea,2.5
47,Olivo,sufficiente,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Olea,2.5
77,Ligustro giapponese,sufficiente,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ligustrum,2.5
124,Ligustro giapponese,sufficiente,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ligustrum,2.5


In [ ]:
# Iterate through each row in bologna_trees
for index, bologna_row in bologna_trees.iterrows():
    if(bologna_trees['crown_width'].isna()):
        # Find the corresponding species in Vienna dataset (Closest_Species)
        family_species = bologna_row['Specie arborea'].split()[0]
        
        # Filter Vienna dataset to find matching trees
        matching_trees_messina = trees_Messina[trees_Messina['Family'] == family_species]
        
        # Check if there are matching trees
        if not matching_trees_messina.empty:
            # Calculate the absolute height difference
            height_diff = np.abs(matching_trees_messina['total_height'] - bologna_row['total_height'])
            
            # Find the index of the closest tree
            closest_index = height_diff.idxmin()

            # Update 'crown_width' for the current row with the value from Vienna dataset
            # if closest_index is nan, skip
            if pd.isna(closest_index) or matching_trees_messina.at[closest_index, 'DIAMETRO CORONA_TXT'] == 'nicht bekannt':
                continue
            else:
                bologna_trees.at[index, 'crown_width'] = matching_trees_messina.at[closest_index, 'DIAMETRO CORONA_TXT']

In [ ]:
# now do the same for trees_Messina
trees_Messina['Family'] = trees_Messina['SPECIE_SC'].str.split().str[0]

match_Messina = []
for no_match_family in no_matching_species_df['Family'].unique():
    # check if the family name is in the column 'Family' of trees_Messina
    if no_match_family in trees_Messina['Family'].unique():
        # add the family name to the list
        match_Messina.append(no_match_family)
        # print the family name
        print(no_match_family)

Olea
Melia
Laurus
Ligustrum
Eriobotrya
Acacia
Nerium
Pittosporum
Yucca
Washingtonia


In [ ]:
# merge the two lists, without duplicates
match = list(set(match_Ferrara + match_Messina))
match

['Washingtonia',
 'Acacia',
 'Libocedrus',
 'Laurus',
 'Melia',
 'Viburnum',
 'Olea',
 'Ligustrum',
 'Nerium',
 'Eriobotrya',
 'Yucca',
 'Pittosporum',
 'Punica',
 'Diospyros']

In [ ]:
# find the unique values for the family names of the no matching species
no_matching_species_df['Family'] = no_matching_species_df['Species_Bologna'].str.split().str[0]
no_matching_species_df['Family'].unique()

# find the value that are in no_matching_species_df['Family'] but not in match
no_match = []
for family in no_matching_species_df['Family'].unique():
    if family not in match:
        no_match.append(family)
        print(family)

Lagestroemia
Zyziphus
Sterculia
nan
Kalmia
Pyracantha
Arbutus
Forsythia
Bignonia
Osmanthus
Eleagnus
Chimonanthus
Chamaerops
Chaenomeles
Trachicarpus
Vite
Vitex
Phyllirea
Actinidia
Deutzia
